In [7]:
import random

import simpy

In [8]:
RANDOM_SEED = 16
NUM_MACHINES = 2  
WASHTIME = 30     
T_INTER = 42       
SIM_TIME = 120

When  persons arrive at the laundry, they requests a machine. Once clothes got one, it starts the laundry’s wash processes and waits for it to finish. It finally releases the machine and people will take them away.

The persons are generated by a setup process. After creating an intial amount of clothes, it creates new washing processes after a random time interval as long as the simulation continues.

In [16]:
class Laundry(object):

    def __init__(self, env, num_machines, washtime):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.washtime = washtime

    def wash(self, person):
        yield self.env.timeout(WASHTIME)
        print("Laundry removed %d%% of %s's dirt." %
              (random.randint(50, 99), person))


The process (each person has a ``name``) arrives at the carwash (``cw``) and requests a cleaning machine. 
It then starts the washing process, waits for it to finish and leaves to never come back.

In [17]:
def person(env, name, cw):
    
    print('%s arrives at the Laundry at %.2f.' % (name, env.now))
    with cw.machine.request() as request:
        yield request

        print('%s enters the Laundry at %.2f.' % (name, env.now))
        yield env.process(cw.wash(name))

        print('%s leaves the Laundry at %.2f.' % (name, env.now))


In [18]:
def setup(env, num_machines, washtime, t_inter):

    laundry = Laundry(env, num_machines, washtime)

    for i in range(4):
        env.process(person(env, 'Person %d' % i, laundry))
        
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1
        env.process(person(env, 'Person %d' % i, laundry))

In [19]:
print('Laundry')

random.seed(RANDOM_SEED)  

env = simpy.Environment()
env.process(setup(env, NUM_MACHINES, WASHTIME, T_INTER))

env.run(until=SIM_TIME)

Laundry
Person 0 arrives at the Laundry at 0.00.
Person 1 arrives at the Laundry at 0.00.
Person 2 arrives at the Laundry at 0.00.
Person 3 arrives at the Laundry at 0.00.
Person 0 enters the Laundry at 0.00.
Person 1 enters the Laundry at 0.00.
Laundry removed 80% of Person 0's dirt.
Laundry removed 80% of Person 1's dirt.
Person 0 leaves the Laundry at 30.00.
Person 1 leaves the Laundry at 30.00.
Person 2 enters the Laundry at 30.00.
Person 3 enters the Laundry at 30.00.
Person 4 arrives at the Laundry at 42.00.
Laundry removed 76% of Person 2's dirt.
Laundry removed 64% of Person 3's dirt.
Person 2 leaves the Laundry at 60.00.
Person 3 leaves the Laundry at 60.00.
Person 4 enters the Laundry at 60.00.
Person 5 arrives at the Laundry at 84.00.
Person 5 enters the Laundry at 84.00.
Laundry removed 50% of Person 4's dirt.
Person 4 leaves the Laundry at 90.00.
Laundry removed 76% of Person 5's dirt.
Person 5 leaves the Laundry at 114.00.
